# Comparaison Multi-Modeles de Generation Video

**Module :** 03-Video-Orchestration  
**Niveau :** Avance  
**Technologies :** HunyuanVideo vs LTX-Video vs Wan vs SVD (benchmark)  
**Duree estimee :** 60 minutes  
**VRAM :** ~18 GB (chargement sequentiel)  

## Objectifs d'Apprentissage

- [ ] Mettre en place un framework de benchmark pour modeles video generatifs
- [ ] Charger et decharger les modeles sequentiellement (gestion VRAM)
- [ ] Generer des videos avec le meme prompt sur chaque modele
- [ ] Mesurer les metriques : temps de generation, VRAM, coherence temporelle, qualite
- [ ] Produire une grille comparative de frames cote-a-cote
- [ ] Analyser le compromis cout/qualite entre modeles

## Prerequis

- GPU avec 18+ GB VRAM (RTX 3090 / RTX 4090)
- Module 02-Advanced complete (notebooks 02-1 a 02-4)
- Packages : `diffusers>=0.32`, `transformers`, `torch`, `accelerate`, `bitsandbytes`, `imageio`, `imageio-ffmpeg`

**Navigation :** [<< 02-4](../02-Advanced/02-4-SVD-Image-to-Video.ipynb) | [Index](../README.md) | [Suivant >>](03-2-Video-Workflow-Orchestration.ipynb)

In [1]:
# Parametres Papermill - JAMAIS modifier ce commentaire

# Configuration notebook
notebook_mode = "interactive"        # "interactive" ou "batch"
skip_widgets = False               # True pour mode batch MCP
debug_level = "INFO"

# Parametres benchmark
models_to_test = ["hunyuan", "ltx", "wan"]  # Modeles a comparer
benchmark_prompt = "a cat walking gracefully through a garden, soft sunlight, cinematic"  # Prompt commun
num_frames = 16                    # Nombre de frames par modele
device = "cuda"                    # Device de calcul

# Parametres generation communs
num_inference_steps = 25           # Etapes de debruitage
guidance_scale = 6.0               # CFG scale
height = 320                       # Hauteur video
width = 512                        # Largeur video
fps_output = 8                     # FPS de la video de sortie
seed = 42                          # Graine pour reproductibilite

# Configuration
run_benchmark = True               # Executer le benchmark
save_as_mp4 = True                 # Sauvegarder en MP4
save_results = True

In [2]:
# Parameters
notebook_mode = "batch"
skip_widgets = True


In [3]:
# Setup environnement et imports
import os
import sys
import json
import time
import gc
import warnings
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import logging

warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Import helpers GenAI
GENAI_ROOT = Path.cwd()
while GENAI_ROOT.name != 'GenAI' and len(GENAI_ROOT.parts) > 1:
    GENAI_ROOT = GENAI_ROOT.parent

HELPERS_PATH = GENAI_ROOT / 'shared' / 'helpers'
if HELPERS_PATH.exists():
    sys.path.insert(0, str(HELPERS_PATH.parent))
    try:
        from helpers.genai_helpers import setup_genai_logging
        print("Helpers GenAI importes")
    except ImportError:
        print("Helpers GenAI non disponibles - mode autonome")

OUTPUT_DIR = GENAI_ROOT / 'outputs' / 'benchmark_video'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

logging.basicConfig(level=getattr(logging, debug_level))
logger = logging.getLogger('benchmark_video')

print(f"Comparaison Multi-Modeles de Generation Video")
print(f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Mode : {notebook_mode}")
print(f"Modeles a tester : {models_to_test}")
print(f"Prompt : {benchmark_prompt[:60]}...")
print(f"Frames : {num_frames}, Steps : {num_inference_steps}, CFG : {guidance_scale}")

Helpers GenAI importes
Comparaison Multi-Modeles de Generation Video
Date : 2026-02-26 08:10:41
Mode : batch
Modeles a tester : ['hunyuan', 'ltx', 'wan']
Prompt : a cat walking gracefully through a garden, soft sunlight, ci...
Frames : 16, Steps : 25, CFG : 6.0


In [4]:
# Chargement .env et verification GPU
from dotenv import load_dotenv

current_path = Path.cwd()
found_env = False
for _ in range(4):
    env_path = current_path / '.env'
    if env_path.exists():
        load_dotenv(env_path)
        print(f"Fichier .env charge depuis : {env_path}")
        found_env = True
        break
    current_path = current_path.parent

if not found_env:
    print("Aucun fichier .env trouve")

# Verification GPU
print("\n--- VERIFICATION GPU ---")
print("=" * 40)

import torch

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    vram_total = torch.cuda.get_device_properties(0).total_mem / 1024**3
    vram_free = (torch.cuda.get_device_properties(0).total_mem - torch.cuda.memory_allocated(0)) / 1024**3
    
    print(f"GPU : {gpu_name}")
    print(f"VRAM totale : {vram_total:.1f} GB")
    print(f"VRAM libre : {vram_free:.1f} GB")
    print(f"CUDA : {torch.version.cuda}")
    
    if vram_total < 18:
        print(f"\nAttention : VRAM ({vram_total:.0f} GB) < 18 GB recommandes")
        print("Certains modeles pourraient ne pas charger")
        height = 256
        width = 384
        num_frames = 12
        print(f"  Resolution reduite a {width}x{height}, {num_frames} frames")
else:
    print("CUDA non disponible.")
    print("Le benchmark necessite un GPU. Le notebook montrera le code sans executer.")
    run_benchmark = False
    device = "cpu"

# Verification des dependances
print("\n--- VERIFICATION DEPENDANCES ---")
print("=" * 40)

deps_ok = True

try:
    import diffusers
    print(f"diffusers : v{diffusers.__version__}")
except ImportError:
    print("diffusers NON INSTALLE (pip install diffusers>=0.32)")
    deps_ok = False

try:
    import transformers
    print(f"transformers : v{transformers.__version__}")
except ImportError:
    print("transformers NON INSTALLE")
    deps_ok = False

try:
    import imageio
    print(f"imageio : v{imageio.__version__}")
except ImportError:
    print("imageio NON INSTALLE")
    deps_ok = False

if not deps_ok:
    print("\nDependances manquantes. Le notebook montrera le code sans executer.")
    run_benchmark = False

print(f"\nDevice : {device}")
print(f"Benchmark active : {run_benchmark}")

Fichier .env charge depuis : D:\Dev\CoursIA.worktrees\GenAI_Series\MyIA.AI.Notebooks\GenAI\.env

--- VERIFICATION GPU ---


CUDA non disponible.
Le benchmark necessite un GPU. Le notebook montrera le code sans executer.

--- VERIFICATION DEPENDANCES ---


diffusers : v0.36.0
transformers NON INSTALLE
imageio : v2.37.2

Dependances manquantes. Le notebook montrera le code sans executer.

Device : cpu
Benchmark active : False


## Section 1 : Framework de benchmark

Pour comparer equitablement les modeles, nous definissons un framework commun
qui garantit les memes conditions de test pour chacun.

| Modele | Type | VRAM estimee | Pipeline diffusers |
|--------|------|-------------|--------------------|
| **HunyuanVideo** | Text-to-video | ~18 GB (INT8) | `HunyuanVideoPipeline` |
| **LTX-Video** | Text-to-video | ~8 GB | `LTXPipeline` |
| **Wan** | Text-to-video | ~10 GB | `WanPipeline` |
| **SVD** | Image-to-video | ~10 GB | `StableVideoDiffusionPipeline` |

### Strategie de gestion memoire

Les modeles sont charges et decharges **sequentiellement** pour respecter les
contraintes VRAM d'une seule carte GPU (24 GB). Apres chaque modele, nous
liberons explicitement la memoire GPU avec `torch.cuda.empty_cache()`.

In [5]:
# Framework de benchmark
print("\n--- FRAMEWORK DE BENCHMARK ---")
print("=" * 45)

from diffusers.utils import export_to_video

# Registre des modeles supportes
MODEL_REGISTRY = {
    "hunyuan": {
        "name": "HunyuanVideo",
        "model_id": "tencent/HunyuanVideo",
        "pipeline_class": "HunyuanVideoPipeline",
        "type": "text-to-video",
        "vram_estimate_gb": 18,
        "quantize": True,
        "supports_negative_prompt": True
    },
    "ltx": {
        "name": "LTX-Video",
        "model_id": "Lightricks/LTX-Video",
        "pipeline_class": "LTXPipeline",
        "type": "text-to-video",
        "vram_estimate_gb": 8,
        "quantize": False,
        "supports_negative_prompt": True
    },
    "wan": {
        "name": "Wan 2.1",
        "model_id": "Wan-AI/Wan2.1-T2V-14B-Diffusers",
        "pipeline_class": "WanPipeline",
        "type": "text-to-video",
        "vram_estimate_gb": 10,
        "quantize": False,
        "supports_negative_prompt": True
    },
    "svd": {
        "name": "SVD 1.1 XT",
        "model_id": "stabilityai/stable-video-diffusion-img2vid-xt",
        "pipeline_class": "StableVideoDiffusionPipeline",
        "type": "image-to-video",
        "vram_estimate_gb": 10,
        "quantize": False,
        "supports_negative_prompt": False
    }
}


def release_vram():
    """Libere la VRAM GPU de facon aggressive."""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
        vram_free = (torch.cuda.get_device_properties(0).total_mem - torch.cuda.memory_allocated(0)) / 1024**3
        print(f"  VRAM liberee. Disponible : {vram_free:.1f} GB")


def measure_temporal_coherence(frames: List) -> Dict[str, float]:
    """
    Mesure la coherence temporelle entre les frames generees.
    
    Metriques :
    - avg_diff : difference moyenne inter-frames (mouvement global)
    - std_diff : ecart-type des differences (regularite du mouvement)
    - max_diff : saut maximal (detecte les artefacts temporels)
    """
    diffs = []
    for i in range(len(frames) - 1):
        f1 = np.array(frames[i]).astype(float)
        f2 = np.array(frames[i + 1]).astype(float)
        diff = np.mean(np.abs(f1 - f2))
        diffs.append(diff)
    
    return {
        "avg_diff": float(np.mean(diffs)),
        "std_diff": float(np.std(diffs)),
        "max_diff": float(np.max(diffs)),
        "stability": "Haute" if np.mean(diffs) < 15 else "Moyenne" if np.mean(diffs) < 30 else "Basse"
    }


def measure_frame_quality(frames: List) -> Dict[str, float]:
    """
    Mesure des indicateurs de qualite sur les frames individuelles.
    
    Metriques :
    - avg_sharpness : nettete moyenne (variance du Laplacien)
    - color_diversity : diversite chromatique (ecart-type couleurs)
    """
    from PIL import ImageFilter
    
    sharpness_scores = []
    color_scores = []
    
    for frame in frames:
        arr = np.array(frame).astype(float)
        # Nettete via variance du laplacien approxime
        gray = np.mean(arr, axis=2)
        laplacian = np.abs(gray[:-2, 1:-1] + gray[2:, 1:-1] + gray[1:-1, :-2] + gray[1:-1, 2:] - 4 * gray[1:-1, 1:-1])
        sharpness_scores.append(float(np.var(laplacian)))
        # Diversite chromatique
        color_scores.append(float(np.std(arr)))
    
    return {
        "avg_sharpness": float(np.mean(sharpness_scores)),
        "color_diversity": float(np.mean(color_scores))
    }


# Afficher les modeles disponibles
print(f"Modeles demandes : {models_to_test}")
print(f"\nRegistre des modeles :")
for key, info in MODEL_REGISTRY.items():
    status = "SELECTIONNE" if key in models_to_test else "disponible"
    print(f"  {key:<10} : {info['name']:<20} ({info['type']}, ~{info['vram_estimate_gb']} GB) [{status}]")


--- FRAMEWORK DE BENCHMARK ---
Modeles demandes : ['hunyuan', 'ltx', 'wan']

Registre des modeles :
  hunyuan    : HunyuanVideo         (text-to-video, ~18 GB) [SELECTIONNE]
  ltx        : LTX-Video            (text-to-video, ~8 GB) [SELECTIONNE]
  wan        : Wan 2.1              (text-to-video, ~10 GB) [SELECTIONNE]
  svd        : SVD 1.1 XT           (image-to-video, ~10 GB) [disponible]


### Interpretation : Framework de benchmark

| Composant | Role | Justification |
|-----------|------|---------------|
| `MODEL_REGISTRY` | Catalogue des modeles | Parametres specifiques a chaque modele |
| `release_vram()` | Liberation memoire | Essentiel pour le chargement sequentiel |
| `measure_temporal_coherence()` | Qualite temporelle | Detecte les sauts et la regularite |
| `measure_frame_quality()` | Qualite spatiale | Nettete et richesse chromatique |

**Points cles** :
1. Le chargement sequentiel est obligatoire car aucun GPU grand public ne peut contenir tous les modeles simultanement
2. La liberation aggressive de VRAM (`gc.collect()` + `empty_cache()` + `synchronize()`) est necessaire entre chaque modele
3. Les metriques automatiques ne remplacent pas l'evaluation visuelle mais permettent un classement objectif

## Section 2 : Execution du benchmark

Nous allons charger chaque modele, generer une video avec le meme prompt,
mesurer les performances puis decharger le modele avant de passer au suivant.

In [6]:
# Execution du benchmark sequentiel
benchmark_results = []

if run_benchmark:
    print("\n--- EXECUTION DU BENCHMARK ---")
    print("=" * 50)
    print(f"Prompt : {benchmark_prompt}")
    print(f"Parametres : {num_frames} frames, {num_inference_steps} steps, CFG={guidance_scale}")
    print(f"Resolution : {width}x{height}")
    
    for model_key in models_to_test:
        if model_key not in MODEL_REGISTRY:
            print(f"\nModele inconnu : {model_key} (ignore)")
            continue
        
        model_info = MODEL_REGISTRY[model_key]
        print(f"\n{'='*50}")
        print(f"Modele : {model_info['name']} ({model_key})")
        print(f"Type : {model_info['type']}")
        print(f"VRAM estimee : ~{model_info['vram_estimate_gb']} GB")
        print(f"{'='*50}")
        
        result = {
            "model_key": model_key,
            "model_name": model_info['name'],
            "type": model_info['type'],
            "success": False
        }
        
        try:
            # 1. Chargement du pipeline
            print(f"  Chargement du pipeline...")
            start_load = time.time()
            
            if model_key == "hunyuan":
                from diffusers import HunyuanVideoPipeline, BitsAndBytesConfig
                quant_config = BitsAndBytesConfig(load_in_8bit=True)
                pipe = HunyuanVideoPipeline.from_pretrained(
                    model_info['model_id'],
                    quantization_config=quant_config,
                    torch_dtype=torch.float16
                )
            elif model_key == "ltx":
                from diffusers import LTXPipeline
                pipe = LTXPipeline.from_pretrained(
                    model_info['model_id'],
                    torch_dtype=torch.float16
                )
            elif model_key == "wan":
                from diffusers import WanPipeline
                pipe = WanPipeline.from_pretrained(
                    model_info['model_id'],
                    torch_dtype=torch.float16
                )
            elif model_key == "svd":
                from diffusers import StableVideoDiffusionPipeline
                pipe = StableVideoDiffusionPipeline.from_pretrained(
                    model_info['model_id'],
                    torch_dtype=torch.float16,
                    variant="fp16"
                )
            
            pipe = pipe.to(device)
            pipe.enable_vae_slicing()
            try:
                pipe.enable_model_cpu_offload()
            except Exception:
                pass
            
            load_time = time.time() - start_load
            result["load_time"] = load_time
            print(f"  Pipeline charge en {load_time:.1f}s")
            
            if device == "cuda":
                vram_after_load = torch.cuda.memory_allocated(0) / 1024**3
                result["vram_model"] = vram_after_load
                print(f"  VRAM apres chargement : {vram_after_load:.1f} GB")
            
            # 2. Generation
            print(f"  Generation en cours...")
            generator = torch.Generator(device=device).manual_seed(seed)
            if device == "cuda":
                torch.cuda.reset_peak_memory_stats()
            
            start_gen = time.time()
            
            if model_info['type'] == "text-to-video":
                gen_kwargs = {
                    "prompt": benchmark_prompt,
                    "num_frames": num_frames,
                    "guidance_scale": guidance_scale,
                    "num_inference_steps": num_inference_steps,
                    "height": height,
                    "width": width,
                    "generator": generator
                }
                if model_info['supports_negative_prompt']:
                    gen_kwargs["negative_prompt"] = "low quality, blurry, distorted, watermark"
                output = pipe(**gen_kwargs)
            else:
                # SVD : image-to-video, creer une image placeholder
                from PIL import ImageDraw
                placeholder = Image.new('RGB', (1024, 576), (100, 150, 200))
                draw = ImageDraw.Draw(placeholder)
                for y in range(576):
                    t = y / 576
                    r = int(135 + 80 * t)
                    g = int(180 - 40 * t)
                    b = int(220 - 60 * t)
                    draw.line([(0, y), (1024, y)], fill=(r, g, b))
                
                output = pipe(
                    image=placeholder,
                    num_frames=min(num_frames, 25),
                    fps=7,
                    motion_bucket_id=127,
                    noise_aug_strength=0.02,
                    num_inference_steps=num_inference_steps,
                    decode_chunk_size=8,
                    generator=generator
                )
            
            gen_time = time.time() - start_gen
            frames = output.frames[0]
            
            result["generation_time"] = gen_time
            result["time_per_frame"] = gen_time / len(frames)
            result["num_frames_actual"] = len(frames)
            result["frames"] = frames
            result["success"] = True
            
            if device == "cuda":
                result["vram_peak"] = torch.cuda.max_memory_allocated(0) / 1024**3
            
            # Metriques de qualite
            result["coherence"] = measure_temporal_coherence(frames)
            result["quality"] = measure_frame_quality(frames)
            
            print(f"  Generation reussie en {gen_time:.1f}s ({len(frames)} frames)")
            if "vram_peak" in result:
                print(f"  VRAM pic : {result['vram_peak']:.1f} GB")
            print(f"  Coherence temporelle : {result['coherence']['stability']}")
            
            # Sauvegarde MP4
            if save_as_mp4:
                mp4_path = OUTPUT_DIR / f"benchmark_{model_key}.mp4"
                export_to_video(frames, str(mp4_path), fps=fps_output)
                result["mp4_path"] = str(mp4_path)
                print(f"  MP4 : {mp4_path.name}")
            
        except Exception as e:
            result["error"] = f"{type(e).__name__}: {str(e)[:200]}"
            print(f"  Erreur : {result['error']}")
        
        finally:
            # 3. Liberation VRAM
            print(f"  Liberation VRAM...")
            try:
                del pipe
            except NameError:
                pass
            release_vram()
        
        benchmark_results.append(result)
    
    print(f"\nBenchmark termine : {sum(1 for r in benchmark_results if r['success'])}/{len(benchmark_results)} modeles reussis")

else:
    print("Benchmark desactive")
    print("\nLe benchmark charge chaque modele sequentiellement :")
    print("  1. Charger le pipeline")
    print("  2. Generer avec le prompt commun")
    print("  3. Mesurer les metriques")
    print("  4. Liberer la VRAM")
    print("  5. Passer au modele suivant")

Benchmark desactive

Le benchmark charge chaque modele sequentiellement :
  1. Charger le pipeline
  2. Generer avec le prompt commun
  3. Mesurer les metriques
  4. Liberer la VRAM
  5. Passer au modele suivant


### Interpretation : Resultats du benchmark

Chaque modele a ete charge puis decharge sequentiellement.
Les metriques collectees permettent une comparaison objective.

**Metriques mesurees** :
- **Temps de chargement** : duree pour telecharger/charger les poids du modele
- **Temps de generation** : duree de l'inference seule
- **VRAM pic** : consommation memoire GPU maximale pendant la generation
- **Coherence temporelle** : regularite du mouvement entre frames consecutives
- **Nettete** : variance du Laplacien sur les frames individuelles

## Section 3 : Visualisation comparative

Nous allons maintenant produire une grille visuelle montrant les frames
de chaque modele cote-a-cote pour permettre une evaluation qualitative.

In [7]:
# Visualisation comparative : grille de frames
successful_results = [r for r in benchmark_results if r.get('success', False)]

if successful_results:
    print("\n--- GRILLE COMPARATIVE ---")
    print("=" * 40)
    
    n_models = len(successful_results)
    n_preview = 4  # Frames par modele
    
    fig, axes = plt.subplots(n_models, n_preview, figsize=(3.5 * n_preview, 3 * n_models))
    if n_models == 1:
        axes = [axes]
    
    for v_idx, result in enumerate(successful_results):
        frames = result['frames']
        frame_indices = np.linspace(0, len(frames) - 1, n_preview, dtype=int)
        
        for f_idx, fi in enumerate(frame_indices):
            axes[v_idx][f_idx].imshow(frames[fi])
            axes[v_idx][f_idx].axis('off')
            if f_idx == 0:
                axes[v_idx][f_idx].set_ylabel(
                    result['model_name'], fontsize=11, fontweight='bold'
                )
            axes[v_idx][f_idx].set_title(f"Frame {fi + 1}", fontsize=8)
    
    plt.suptitle(
        f"Benchmark : {benchmark_prompt[:50]}...",
        fontsize=13, fontweight='bold'
    )
    plt.tight_layout()
    plt.show()
    
    # Courbe de difference inter-frames pour chaque modele
    fig, ax = plt.subplots(figsize=(12, 5))
    for result in successful_results:
        frames = result['frames']
        diffs = []
        for i in range(len(frames) - 1):
            f1 = np.array(frames[i]).astype(float)
            f2 = np.array(frames[i + 1]).astype(float)
            diffs.append(np.mean(np.abs(f1 - f2)))
        ax.plot(range(1, len(diffs) + 1), diffs, marker='o', label=result['model_name'], linewidth=2)
    
    ax.set_xlabel('Transition entre frames', fontsize=11)
    ax.set_ylabel('Difference moyenne (pixels)', fontsize=11)
    ax.set_title('Coherence temporelle par modele', fontsize=13, fontweight='bold')
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

else:
    print("Aucun resultat de benchmark disponible pour la visualisation")
    print("\nEn mode benchmark, cette cellule affiche :")
    print("  1. Grille de frames (modeles en lignes, frames en colonnes)")
    print("  2. Courbe de coherence temporelle")

Aucun resultat de benchmark disponible pour la visualisation

En mode benchmark, cette cellule affiche :
  1. Grille de frames (modeles en lignes, frames en colonnes)
  2. Courbe de coherence temporelle


### Interpretation : Analyse visuelle

La grille comparative permet d'evaluer visuellement :

| Critere | Meilleur modele typique | Observation |
|---------|----------------------|-------------|
| Nettete des details | HunyuanVideo | Architecture Transformer 3D plus precise |
| Fluidite du mouvement | HunyuanVideo / Wan | Flow matching plus regulier |
| Vitesse de generation | LTX-Video | Architecture optimisee pour la vitesse |
| Economie VRAM | LTX-Video | ~8 GB contre ~18 GB pour HunyuanVideo |
| Fidelite image source | SVD | Specialise image-to-video |

**Points cles** :
1. La courbe de coherence temporelle revele les modeles avec des sauts brusques (artefacts)
2. Un mouvement trop faible (diff ~0) indique un modele qui genere des frames quasi-statiques
3. Un mouvement trop fort (diff > 40) indique une instabilite temporelle

## Section 4 : Tableau recapitulatif et analyse cout/qualite

In [8]:
# Tableau recapitulatif et analyse cout/qualite
if successful_results:
    print("\n--- TABLEAU RECAPITULATIF ---")
    print("=" * 90)
    
    # En-tete
    header = f"{'Modele':<18} {'Type':<16} {'Chargement':<12} {'Generation':<12} {'VRAM pic':<11} {'Coherence':<12} {'Nettete':<10}"
    print(header)
    print("-" * 90)
    
    for r in successful_results:
        load_t = f"{r.get('load_time', 0):.0f}s"
        gen_t = f"{r.get('generation_time', 0):.1f}s"
        vram = f"{r.get('vram_peak', 0):.1f} GB"
        coherence = r.get('coherence', {}).get('stability', 'N/A')
        sharpness = f"{r.get('quality', {}).get('avg_sharpness', 0):.0f}"
        
        print(f"  {r['model_name']:<18} {r['type']:<16} {load_t:<12} {gen_t:<12} {vram:<11} {coherence:<12} {sharpness:<10}")
    
    # Analyse cout/qualite
    print(f"\n--- ANALYSE COUT / QUALITE ---")
    print("=" * 60)
    
    # Score composite : qualite / (temps * vram)
    print(f"\n{'Modele':<18} {'Score efficacite':<20} {'Recommandation':<30}")
    print("-" * 68)
    
    for r in successful_results:
        gen_time = r.get('generation_time', 1)
        vram = r.get('vram_peak', 1)
        sharpness = r.get('quality', {}).get('avg_sharpness', 1)
        coherence_val = r.get('coherence', {}).get('avg_diff', 30)
        
        # Score : nettete / (temps * vram), normalise par coherence
        coherence_factor = 1.0 if coherence_val < 15 else 0.8 if coherence_val < 30 else 0.5
        efficiency = (sharpness * coherence_factor) / (gen_time * vram) * 100
        
        if efficiency > 1.0:
            reco = "Excellent rapport qualite/cout"
        elif efficiency > 0.3:
            reco = "Bon compromis"
        else:
            reco = "Qualite elevee, cout important"
        
        print(f"  {r['model_name']:<18} {efficiency:<20.2f} {reco:<30}")
    
    # Sauvegarde du rapport JSON
    if save_results:
        report = {
            "timestamp": datetime.now().isoformat(),
            "prompt": benchmark_prompt,
            "params": {
                "num_frames": num_frames,
                "num_inference_steps": num_inference_steps,
                "guidance_scale": guidance_scale,
                "height": height,
                "width": width
            },
            "results": [
                {
                    "model": r['model_name'],
                    "type": r['type'],
                    "load_time": r.get('load_time', 0),
                    "generation_time": r.get('generation_time', 0),
                    "vram_peak": r.get('vram_peak', 0),
                    "coherence": r.get('coherence', {}),
                    "quality": r.get('quality', {})
                }
                for r in successful_results
            ]
        }
        report_path = OUTPUT_DIR / "benchmark_report.json"
        with open(report_path, 'w', encoding='utf-8') as f:
            json.dump(report, f, indent=2, ensure_ascii=False)
        print(f"\nRapport sauvegarde : {report_path.name}")

else:
    print("Aucun resultat de benchmark disponible")
    print("\nTableau attendu (valeurs typiques RTX 3090) :")
    print(f"{'Modele':<18} {'Generation':<12} {'VRAM pic':<11} {'Coherence':<12}")
    print("-" * 55)
    print(f"  {'HunyuanVideo':<18} {'60-180s':<12} {'~18 GB':<11} {'Haute':<12}")
    print(f"  {'LTX-Video':<18} {'15-30s':<12} {'~8 GB':<11} {'Moyenne':<12}")
    print(f"  {'Wan 2.1':<18} {'30-90s':<12} {'~10 GB':<11} {'Haute':<12}")
    print(f"  {'SVD 1.1 XT':<18} {'20-40s':<12} {'~10 GB':<11} {'Haute':<12}")

Aucun resultat de benchmark disponible

Tableau attendu (valeurs typiques RTX 3090) :
Modele             Generation   VRAM pic    Coherence   
-------------------------------------------------------
  HunyuanVideo       60-180s      ~18 GB      Haute       
  LTX-Video          15-30s       ~8 GB       Moyenne     
  Wan 2.1            30-90s       ~10 GB      Haute       
  SVD 1.1 XT         20-40s       ~10 GB      Haute       


In [9]:
# Mode interactif : benchmark avec un prompt utilisateur
if notebook_mode == "interactive" and not skip_widgets:
    print("\n--- MODE INTERACTIF ---")
    print("=" * 40)
    print("Entrez votre propre prompt pour tester sur tous les modeles.")
    print("(Laissez vide pour passer a la suite)")
    
    try:
        user_prompt = input("\nVotre prompt : ").strip()
        
        if user_prompt and run_benchmark:
            print(f"\nBenchmark avec : {user_prompt}")
            print("Note : le benchmark complet prendra plusieurs minutes.")
            print("Seul le modele le plus rapide (LTX-Video) sera teste en interactif.")
            
            if "ltx" in MODEL_REGISTRY:
                try:
                    from diffusers import LTXPipeline
                    pipe_user = LTXPipeline.from_pretrained(
                        MODEL_REGISTRY['ltx']['model_id'],
                        torch_dtype=torch.float16
                    ).to(device)
                    pipe_user.enable_vae_slicing()
                    
                    generator = torch.Generator(device=device).manual_seed(seed)
                    output = pipe_user(
                        prompt=user_prompt,
                        negative_prompt="low quality, blurry, distorted",
                        num_frames=num_frames,
                        guidance_scale=guidance_scale,
                        num_inference_steps=num_inference_steps,
                        height=height,
                        width=width,
                        generator=generator
                    )
                    
                    user_frames = output.frames[0]
                    n_display = min(8, len(user_frames))
                    indices = np.linspace(0, len(user_frames) - 1, n_display, dtype=int)
                    fig, axes = plt.subplots(1, n_display, figsize=(2.5 * n_display, 3))
                    if n_display == 1:
                        axes = [axes]
                    for ax, idx in zip(axes, indices):
                        ax.imshow(user_frames[idx])
                        ax.set_title(f"Frame {idx+1}", fontsize=8)
                        ax.axis('off')
                    plt.suptitle(f"LTX-Video : {user_prompt[:50]}...", fontweight='bold')
                    plt.tight_layout()
                    plt.show()
                    
                    del pipe_user
                    release_vram()
                    
                except Exception as e:
                    print(f"Erreur : {type(e).__name__}: {str(e)[:100]}")
        elif user_prompt:
            print("Benchmark non disponible")
        else:
            print("Mode interactif ignore")
    
    except (KeyboardInterrupt, EOFError) as e:
        print(f"\nMode interactif interrompu ({type(e).__name__})")
    except Exception as e:
        error_type = type(e).__name__
        if "StdinNotImplemented" in error_type or "input" in str(e).lower():
            print("\nMode interactif non disponible (execution automatisee)")
        else:
            print(f"\nErreur inattendue : {error_type} - {str(e)[:100]}")
            print("Passage a la suite du notebook")
else:
    print("\nMode batch - Interface interactive desactivee")


Mode batch - Interface interactive desactivee


## Bonnes pratiques et choix de modele

### Guide de selection par cas d'usage

| Cas d'usage | Modele recommande | Raison |
|-------------|------------------|--------|
| Prototypage rapide | LTX-Video | Rapide, leger (~8 GB) |
| Qualite maximale | HunyuanVideo | Meilleure coherence et resolution |
| Prompts multilingues | Wan 2.1 | Support natif du chinois, francais, etc. |
| Animation d'image existante | SVD | Specialise image-to-video |
| Production batch | LTX-Video | Rapport vitesse/qualite optimal |
| Demo client | HunyuanVideo | Qualite visuelle maximale |

### Strategies d'optimisation VRAM

| Strategie | Gain VRAM | Impact qualite |
|-----------|-----------|----------------|
| Quantification INT8 | -40% | Faible |
| VAE slicing | -15% pic | Aucun |
| VAE tiling | -10% pic | Aucun |
| CPU offload | Variable | Augmente le temps |
| Resolution reduite | -30-50% | Proportionnel |

In [10]:
# Statistiques de session et prochaines etapes
print("\n--- STATISTIQUES DE SESSION ---")
print("=" * 40)

print(f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Mode : {notebook_mode}")
print(f"Modeles testes : {models_to_test}")
print(f"Prompt : {benchmark_prompt[:60]}...")
print(f"Parametres : {num_frames} frames, {num_inference_steps} steps, CFG={guidance_scale}")
print(f"Resolution : {width}x{height}")

if benchmark_results:
    n_success = sum(1 for r in benchmark_results if r.get('success', False))
    print(f"\nResultats : {n_success}/{len(benchmark_results)} modeles reussis")
    for r in benchmark_results:
        status = "OK" if r.get('success') else "ERREUR"
        gen_t = f"{r.get('generation_time', 0):.1f}s" if r.get('success') else r.get('error', 'N/A')[:50]
        print(f"  {r['model_name']:<18} [{status}] {gen_t}")

if device == "cuda" and torch.cuda.is_available():
    vram_current = torch.cuda.memory_allocated(0) / 1024**3
    print(f"\nVRAM actuelle : {vram_current:.1f} GB")

if save_results and OUTPUT_DIR.exists():
    generated_files = list(OUTPUT_DIR.glob('*'))
    print(f"\nFichiers generes ({len(generated_files)}) :")
    for f in sorted(generated_files):
        size_kb = f.stat().st_size / 1024
        print(f"  {f.name} ({size_kb:.1f} KB)")

print(f"\n--- PROCHAINES ETAPES ---")
print(f"1. Notebook 03-2 : Orchestration de pipelines (text -> image -> video -> upscale)")
print(f"2. Notebook 03-3 : Workflows ComfyUI pour la generation video")
print(f"3. Module 04 : Applications production (education, creatif, bout-en-bout)")

print(f"\nNotebook 03-1 Comparaison Multi-Modeles termine - {datetime.now().strftime('%H:%M:%S')}")


--- STATISTIQUES DE SESSION ---
Date : 2026-02-26 08:10:47
Mode : batch
Modeles testes : ['hunyuan', 'ltx', 'wan']
Prompt : a cat walking gracefully through a garden, soft sunlight, ci...
Parametres : 16 frames, 25 steps, CFG=6.0
Resolution : 512x320

Fichiers generes (0) :

--- PROCHAINES ETAPES ---
1. Notebook 03-2 : Orchestration de pipelines (text -> image -> video -> upscale)
2. Notebook 03-3 : Workflows ComfyUI pour la generation video
3. Module 04 : Applications production (education, creatif, bout-en-bout)

Notebook 03-1 Comparaison Multi-Modeles termine - 08:10:47
